In [2]:
import tensorflow as tf
import numpy as np


2024-09-30 16:34:21.704624: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-30 16:34:21.713213: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-30 16:34:21.723019: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-30 16:34:21.726036: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-30 16:34:21.733328: I tensorflow/core/platform/cpu_feature_guar

In [3]:
import json
with open("graph.json", "r") as f:
    graph = json.load(f)

In [4]:

def get_Mtrx_and_H0(graph):
    H0 = tf.constant(graph['nodes_feature'])
    A = tf.SparseTensor(indices=np.transpose(graph["A"]),
                    values=np.array([v + 0 if index[0] != index[1] else 1
                                     for v, index in zip(graph["edges_feature"], np.transpose(graph["A"]))], dtype=np.float32), 
                    dense_shape=[max(graph["A"][0])+1, max(graph["A"][1])+1])
    d = tf.sparse.reduce_sum(A, axis=1)
    Mtrx = tf.SparseTensor(indices=np.transpose(graph["A"]),
                    values=np.array([v/d[index[0]] for index, v in zip(A.indices, A.values)], dtype=np.float32), 
                    dense_shape=[max(graph["A"][0])+1, max(graph["A"][1])+1])
    
    
    return Mtrx, H0
    

class GraphSegmenter(tf.Module):
    def __init__(self):
        self.W = tf.Variable(tf.random.uniform(shape=[8, 10]))
        self.activation =  tf.nn.relu

    def __call__(self, graph):
        A, H0 = get_Mtrx_and_H0(graph)
        H1 = self.activation(
            tf.matmul(tf.sparse.sparse_dense_matmul(A, H0), self.W)
        )
        
        return tf.reduce_max(H1, axis=1)

def my_loss(H1_emb, H1_true):
    return tf.reduce_sum(tf.pow(H1_emb-H1_true, 2))

In [5]:
Mtrx, H0 = get_Mtrx_and_H0(graph)

2024-09-30 16:34:25.252966: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [6]:
model = GraphSegmenter()

In [7]:
model(graph)

<tf.Tensor: shape=(629,), dtype=float32, numpy=
array([ 301.63086,  354.6005 ,  395.80386,  402.0905 ,  453.31058,
        468.79474,  512.0568 ,  327.90134,  372.04437,  423.6092 ,
        349.4307 ,  334.49274,  320.86298,  331.21426,  341.0999 ,
        329.3128 ,  365.45547,  371.91678,  378.70587,  408.27512,
        402.0715 ,  401.36295,  441.90912,  468.96436,  458.9706 ,
        450.15213,  563.1437 ,  574.59503,  585.89075,  354.83536,
        416.02533,  423.97952,  454.31693,  523.65735,  402.4533 ,
        429.59418,  418.56238,  429.91953,  422.73148,  373.97534,
        256.64417,  350.84067,  382.02182,  386.6449 ,  414.496  ,
        437.14285,  494.6103 ,  353.4176 ,  410.1336 ,  450.76837,
        502.63275,  338.10327,  359.14407,  396.0119 ,  432.77258,
        459.31018,  505.2959 ,  345.05527,  369.23737,  395.4232 ,
        419.27933,  438.93875,  463.29868,  504.34137,  355.01852,
        380.7302 ,  404.43814,  426.07166,  429.62787,  439.6559 ,
        460.43

In [8]:
with tf.GradientTape() as tape:
    h1 = model(graph)
    loss = my_loss(h1, tf.eye(num_rows=629, num_columns=1))

tape.gradient(loss, model.trainable_variables)

(<tf.Tensor: shape=(8, 10), dtype=float32, numpy=
 array([[0.0000000e+00, 0.0000000e+00, 3.3968835e+09, 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.8134281e+11,
         0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 2.1724602e+10, 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 2.6875875e+11,
         0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000

# Вторая модель

In [9]:
with open("claster.json", "r") as f:
    clust = json.load(f)

In [10]:
clus = tf.constant(np.array(clust["clus"]), dtype=tf.dtypes.float32)

In [44]:
class GraphSegmenter2(tf.Module):
    def __init__(self):
        self.W = tf.Variable(tf.random.uniform(shape=[8, 10]))
        self.W2 = tf.Variable(tf.random.uniform(shape=[8, 10]))
        self.activation =  tf.nn.relu

    def __call__(self, graph):
        A, H0 = get_Mtrx_and_H0(graph)
        HL = tf.matmul(tf.sparse.sparse_dense_matmul(A, H0), self.W)
        HR = tf.matmul(tf.sparse.sparse_dense_matmul(A, H0), self.W2)
        H1 = self.activation(
            tf.matmul(HL, tf.transpose(HR))
        )
        
        return self.activation(H1)

def my_loss(H1_emb, H1_true):
    v1 = H1_emb
    E = tf.eye(H1_true.dense_shape[0])
    v2 = tf.sparse.sparse_dense_matmul(H1_true, E)
    return tf.reduce_sum(tf.pow(v1-v2, 2))

model2 = GraphSegmenter2()

In [ ]:
opt = tf.optimizers.Adam(learning_rate=1.0)
for i in range(50):
    with tf.GradientTape() as tape:
        h1 = model2(graph)
        A, _ = get_Mtrx_and_H0(graph)
        loss = my_loss(h1, A)
    
    dW = tape.gradient(loss, model2.trainable_variables)
    opt.apply_gradients(zip(dW, model2.trainable_variables))

In [ ]:
r = model2(graph)
my_loss(r, A)

In [240]:
with open("graph2.json", "r") as f:
    graph2 = json.load(f)

In [242]:
model2(graph2)

<tf.Tensor: shape=(677,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [237]:
Mtrx, H = get_Mtrx_and_H0(graph)
tf.reduce_max(tf.matmul(tf.sparse.sparse_dense_matmul(Mtrx, H), model2.W), axis=1)

<tf.Tensor: shape=(629,), dtype=float32, numpy=
array([-100.67611 , -121.25881 , -136.58931 , -132.42946 , -154.11803 ,
       -156.46323 , -169.35999 , -110.87806 , -126.728   , -145.87689 ,
       -109.25439 , -111.71451 , -106.963844, -110.732086, -114.31059 ,
       -111.8679  , -123.62674 , -125.728714, -128.24988 , -139.23776 ,
       -136.3829  , -139.3267  , -151.7208  , -161.77507 , -157.51273 ,
       -145.61841 , -188.40791 , -196.52371 , -196.41817 , -111.16015 ,
       -139.03334 , -143.47151 , -154.83267 , -173.30742 , -132.91986 ,
       -144.11055 , -139.51523 , -142.23692 , -134.67084 , -114.85678 ,
        -82.64516 , -104.067604, -117.0756  , -118.7924  , -127.80632 ,
       -137.54512 , -159.18571 , -105.29932 , -124.99538 , -141.45111 ,
       -160.95776 ,  -98.11641 , -105.93004 , -119.19375 , -133.16559 ,
       -143.12733 , -161.76698 ,  -99.501396, -108.48157 , -118.12035 ,
       -126.97947 , -134.3656  , -143.4118  , -160.97939 , -101.91917 ,
       -111.3818

In [220]:
my_loss(model2(graph), clus)

<tf.Tensor: shape=(), dtype=float32, numpy=44371.0>

In [222]:
model2.W

<tf.Variable 'Variable:0' shape=(8, 10) dtype=float32, numpy=
array([[-0.02255897, -0.56373805, -0.23667705, -0.8764673 , -0.0881488 ,
        -0.3762294 ,  0.0786975 , -0.18443012, -0.15318154,  0.04701867],
       [-0.6550233 , -0.2920421 , -0.30421278, -0.16217552, -0.21856076,
        -0.48982003, -0.46877077, -0.45550892, -0.2948032 , -0.4763935 ],
       [ 0.91928804,  0.72769976,  0.8187083 ,  0.13691568,  0.7385869 ,
         0.21124876,  0.08000433,  0.8234364 ,  0.24758875,  0.36636245],
       [ 0.6141236 ,  0.50068104,  0.3811555 ,  0.02555716,  0.45479774,
         0.49741292,  0.37239134,  0.04489887,  0.87694824,  0.19765425],
       [ 0.21644545,  0.3949932 ,  0.5916754 ,  0.8857025 ,  0.18231559,
         0.06700742,  0.02315974,  0.931944  ,  0.49277127,  0.37972426],
       [ 0.72347534,  0.91455245,  0.81568813,  0.68924546,  0.23569977,
         0.6623441 ,  0.11481607,  0.41421223,  0.75635934,  0.8727002 ],
       [ 0.47485054,  0.2408818 ,  0.1656326 ,  0.583086